<a href="https://colab.research.google.com/github/yorkjong/vistock/blob/feature%2Fibd/notebooks/ibd_rs_rating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Stock Analysis and Ranking with IBD RS Rating, inspired by the Investor's Business Daily (IBD) methodology.

### Install and Setup

#### Install Packages

In [1]:
%pip install "git+https://github.com/yorkjong/vistock.git@feature/ibd"
%pip install requests-cache

  Cloning https://github.com/yorkjong/vistock.git (to revision feature/ibd) to /tmp/pip-req-build-_i1ko_ic
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/vistock.git /tmp/pip-req-build-_i1ko_ic
  Running command git checkout -b feature/ibd --track origin/feature/ibd
  Switched to a new branch 'feature/ibd'
  Branch 'feature/ibd' set up to track remote branch 'feature/ibd' from 'origin'.
  Resolved https://github.com/yorkjong/vistock.git to commit c04955db591078b4a0322f84d01018be686919b7
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 875.3 kB/s eta 0:00:00
  Created wheel for vistock: filename=vistock-0.5.0-py3-none-any.whl size=72935 sha256=70ea8e687f8c2afcdff25468ac3b71562db71abf6709cba882e0049213d309b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-7dqlxavj/wheels/07/97/7f/a72c7fbc975be0326ac45fec1ca0c872e2d6e2bbea96caf706
Successfully built vistock
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61

#### Setup and Configuration

In [8]:
# @title Enable DataFrame Formatter
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [9]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [10]:
# @title GitHub
import base64
import requests
import pandas as pd
from io import StringIO


class GitHub:
    def __init__(self, repo_owner, repo_name, token, dir='', branch='main'):
        base = 'https://api.github.com/repos'
        dir = dir.strip('/')
        if dir:
            self.base_url = f'{base}/{repo_owner}/{repo_name}/contents/{dir}'
        else:
            self.base_url = f'{base}/{repo_owner}/{repo_name}/contents'
        self.branch = branch
        self.token = token

    def _request(self, method, url, headers=None, params=None, json=None):
        response = requests.request(method, url, headers=headers,
                                    params=params, json=json)
        if response.status_code in [200, 201]:
            return response.json()
        else:
            print(f"Request failed: {response.status_code} - {response.json()}")
            return None

    def file_exists(self, file_path):
        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json',
        }
        response = requests.get(url, headers=headers,
                                params={'ref': self.branch})
        if response.status_code == 200:
            return True
        elif response.status_code == 404:
            return False
        else:
            print(f"Request failed: {response.status_code} - {response.json()}")
            return None

    def list_filenames(self, dir_path=''):
        url = f'{self.base_url}/{dir_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json',
        }
        response = requests.get(url, headers=headers,
                                params={'ref': self.branch})
        if response.status_code == 200:
            files = response.json()
            return [item['name'] for item in files]
        else:
            print(f"Request failed: {response.status_code} - {response.json()}")
            return []

    def download_file(self, file_path):
        if not self.file_exists(file_path):
            print(f"File '{file_path}' does not exist. Cannot download.")
            return None

        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json',
        }

        file_info = self._request('GET', url, headers=headers)
        if file_info:
            download_url = file_info['download_url']
            response = requests.get(download_url)
            if response.status_code == 200:
                return StringIO(response.text)
            else:
                print(f"Failed to download file: "
                      f"{response.status_code} - {response.text}")
                return None
        return None

    def download_csv(self, file_path):
        file_content = self.download_file(file_path)
        if file_content:
            return pd.read_csv(file_content)
        else:
            return pd.DataFrame()

    def upload_file(self, file_path, content):
        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json'
        }

        encoded_content = base64.b64encode(content.encode()).decode()
        payload = {
            'message': 'Uploading file',
            'content': encoded_content,
            'branch': self.branch
        }
        self._request('PUT', url, headers=headers, json=payload)

    def upload_df_as_csv(self, file_path, df):
        """Upload a DataFrame to a CSV file."""
        if not file_path.endswith('.csv'):
            file_path += '.csv'
        csv_content = df.to_csv(index=False)
        self.upload_file(file_path, csv_content)

    def remove_file(self, file_path):
        if not self.file_exists(file_path):
            print(f"File '{file_path}' does not exist. Skipping deletion.")
            return

        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json'
        }

        # Fetch the file info to get the SHA needed for deletion
        file_info = self._request('GET', url, headers=headers)
        if file_info:
            sha = file_info['sha']
            payload = {
                'message': 'Deleting file',
                'sha': sha,
                'branch': self.branch
            }
            self._request('DELETE', url, headers=headers, json=payload)

#-------------------------------------------------------------------------------

from google.colab import userdata

github = GitHub(
    repo_owner='YorkJong',
    repo_name='stock-reports',
    token=userdata.get('GithubToken.stock-reports'),
    dir='ibd',
)

In [11]:
# @title DataFrame Operations

def is_taiwan_stock_df(df):
    ticker = df['Ticker'].iloc[0].replace('.TWO', '').replace('.TW', '')
    return ticker.isdigit()

def add_name_column(df):
    column_names = df.columns.tolist()
    if 'Name' in column_names:
        return df
    if 'Ticker' not in column_names:
        return df
    if df.empty:
        return df
    if not is_taiwan_stock_df(df):
        return df
    df['Name'] = None
    ticker_index = column_names.index('Ticker')
    column_names.insert(ticker_index + 1, 'Name')
    df = df[column_names]   # create a new DataFrame
    df['Name'] = df['Ticker'].apply(tw.stock_name)
    return df


def update_tickers_with_names(df, ticker_column, name_separator=','):
    """
    Update ticker codes in a DataFrame with their corresponding stock names.

    Parameters:
    - df: The DataFrame containing ticker codes.
    - ticker_column: The name of the column containing ticker codes.
    - name_separator: Separator used to join names (default is comma for multiple tickers).

    This function updates the specified column with the stock names instead of ticker codes.
    """
    # Iterate over the specified column in the DataFrame
    for index, row in df.iterrows():
        tickers = row[ticker_column].split(name_separator)  # Split the tickers string into a list
        stock_names = [tw.stock_name(ticker) for ticker in tickers]  # Get stock names for each ticker
        # Update the stock names back to the DataFrame
        df.at[index, ticker_column] = name_separator.join(stock_names)  # Join the names back into a string


def remove_ticker_suffix(df, ticker_column):
    """
    Remove the '.TW' or '.TWO' suffix from ticker codes in a DataFrame.

    Parameters:
    - df: The DataFrame containing ticker codes.
    - ticker_column: The name of the column containing ticker codes.
    """
    # Apply string replacement for each ticker in the specified column
    df[ticker_column] = df[ticker_column].str.replace('.TWO', '', regex=False)
    df[ticker_column] = df[ticker_column].str.replace('.TW', '', regex=False)


def filter_increasing_relative_strength(df):
    """
    Filter stocks with increasing Relative Strength over different time periods.

    This function filters the DataFrame to include only those stocks where:
    - Relative Strength is above 100.
    - Relative Strength has increased over the past 1 month, 3 months, and 6 months.
    Optionally, you can add a condition to check if Percentile is above 90.
    """
    return df[
        (df["Relative Strength"] > 100)
        & (df["Relative Strength"] > df["1 Month Ago"])
        & (df["1 Month Ago"] > df["3 Months Ago"])
        & (df["3 Months Ago"] > df["6 Months Ago"])
        # & (df["Percentile"] > 90)  # Uncomment to include Percentile filter
    ]


In [12]:
# @title Rank Function

import os
from datetime import datetime

from vistock import ibd
from vistock import tw
from vistock.stock_indices import get_tickers

def remove_failed_tickers(tickers):
    delisted = ['BRK.B', 'LEN.B', 'BF.B', 'UHAL.B', 'BF.A', 'CWEN.A', 'HEI.A']
    invalid = ['GEV', 'SOLV', 'VLTO', 'SW', 'ARM', 'CART', 'AS', 'BIRK', 'VSTS','LOAR', 'ALAB','GRAL', 'SEG']
    return list(set(tickers) - set(delisted) - set(invalid))

def rank(code, period='2y', tickers_getter=get_tickers,
         ticker_ref='^GSPC', out_dir='out'):
    tickers = tickers_getter(code)
    #tickers = [t.lstrip('$') for t in tickers]
    tickers = remove_failed_tickers(tickers)

    rank_stock, rank_indust = ibd.rankings(tickers, period=period,
                                           ticker_ref=ticker_ref)
    if rank_stock.empty or rank_indust.empty:
        print("Not enough data to generate rankings.")
        return

    # Update the stock names back to the DataFrame
    #update_tickers_with_names(rank_stock, 'Ticker')
    #update_tickers_with_names(rank_indust, 'Tickers')

    rank_stock = add_name_column(rank_stock)

    # Remove the '.TW' or '.TWO' suffix
    remove_ticker_suffix(rank_stock, 'Ticker')
    remove_ticker_suffix(rank_indust, 'Tickers')

    # Save to CSV
    print("\n\n***")
    os.makedirs(out_dir, exist_ok=True)
    today = datetime.now().strftime('%Y%m%d')
    for df, kind in zip([rank_stock, rank_indust],
                           ['stocks', 'industries']):
        filename = f'{code}_{kind}_{period}_{today}.csv'
        github.upload_df_as_csv(filename, df)
        df.to_csv(os.path.join(out_dir, filename), index=False)
        print(f'Your "{filename}" is in the "{out_dir}" folder.')
    print("***\n")

    return rank_stock, rank_indust

### Glossary of Terms

source (The source of stocks to analyze):
- This could include stocks traded on exchanges or components of a specific index.
- Common abbreviation(s) for the exchange or market sector.  
  - For Taiwan Markets, possible values include:
    - `TWSE`: Taiwan Stock Exchange (台灣上市股票交易所）
    - `TPEX`: Taipei Exchange （上櫃交易所）
    - `ESB`: Emerging Stock Board （興櫃交易所）
  - Can also be combined with '+' (e.g., `TWSE+TPEX`, `TWSE+TPEX+ESB`)
  - For America Markets, possible values include:
    - `SPX`: S&P 500 (標普五百指數)
    - `DJIA`: Dow Jones Industrial Average (道瓊指數)
    - `NDX`: NASDAQ-100 (納斯達克一百指數)
    - `SOX`: PHLX Semiconductor Index （費半指數）
  - Multiple indices can be combined using '+' (e.g., `SPX+DJIA+NDX+SOX`)

period (Historical Data Time Range)：
- The time range for which to fetch historical data.
- `2y` means 2 years
- `6mo` means 6 monthes

RS (Relative Strength)
- Relative Strength (RS) is a metric used to evaluate the performance of a stock relative to a benchmark index.
  - A higher RS rating indicates that the stock has outperformed the index, while a lower RS rating suggests underperformance.
- The IBD RS calculates the performance of the last year, with the most recent quarter weighted double.

min_percentile (最小百分位)
- The minimum percentile for a stock to be included in the rankings.

### RS Rating and Ranking

In [7]:
source = "PHLX Semiconductor" #@param ["S&P 500", "Dow Jones Industrial Average", "NASDAQ 100", "Russell 1000", "PHLX Semiconductor", "All Indices"]
period = "2y" # @param ["6mo","1y","ytd","2y"]

code_from_name = {
    'S&P 500': 'SPX',
    'Dow Jones Industrial Average': 'DJIA',
    'NASDAQ 100': 'NDX',
    'Russell 1000': 'RUI',
    'PHLX Semiconductor': 'SOX',
    'All Indices': 'SPX+DJIA+NDX+RUI+SOX',
}

rank_stock, rank_indust = rank(code_from_name[source], period)
for df in (rank_stock, rank_indust):
    display(data_table.DataTable(df, include_index=False, num_rows_per_page=10))

[*********************100%***********************]  31 of 31 completed


..............................


***
Your "SOX_stocks_2y_20240827.csv" is in the "out" folder.
Your "SOX_industries_2y_20240827.csv" is in the "out" folder.
***



,Ticker,Price,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
8,NVDA,126.46,Technology,Semiconductors,161.13,167.54,184.46,184.67,99,99,96,99,1
7,COHR,79.43,Technology,Scientific & Technical Instruments,147.45,136.97,127.10,138.16,96,96,82,89,2
14,MPWR,912.25,Technology,Semiconductors,125.75,127.52,118.34,125.00,93,93,68,79,3
23,AVGO,159.62,Technology,Semiconductors,121.86,121.46,127.10,142.79,89,86,82,93,4
9,TSM,169.07,Technology,Semiconductors,121.02,124.28,131.57,118.39,86,89,86,68,5
18,KLAC,798.97,Technology,Semiconductor Equipment & Materials,110.39,120.88,125.47,124.79,82,82,75,75,6
22,TXN,208.25,Technology,Semiconductors,103.96,104.78,106.17,86.18,79,68,41,10,7
19,TER,133.00,Technology,Semiconductor Equipment & Materials,102.59,103.43,127.01,89.89,75,62,79,31,8
1,NOVT,182.54,Technology,Scientific & Technical Instruments,101.61,103.28,92.32,88.57,72,58,34,20,9
13,ADI,225.61,Technology,Semiconductors,98.72,104.03,111.39,89.59,68,65,51,27,10


,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
1,Scientific & Technical Instruments,Technology,124.53,120.12,109.71,113.36,"COHR,NOVT",99,99,49,99,1
0,Semiconductors,Technology,92.86,99.02,108.09,106.34,"NVDA,MPWR,AVGO,TSM,TXN,ADI,QRVO,QCOM,SWKS,MU,N...",49,0,0,0,2
2,Semiconductor Equipment & Materials,Technology,91.39,101.49,111.85,111.78,"KLAC,TER,ASML,AMAT,LRCX,ENTG,IPGP",0,49,99,49,3


In [ ]:
# @title Top Percentile Stocks
min_percentile = 90 # @param {"type":"slider","min":1,"max":99,"step":1}
top_stocks = rank_stock[rank_stock[ibd.TITLE_PERCENTILE] >= min_percentile]
num_rows, _ = top_stocks.shape
print(f'\nnumber of filtered tickers: {num_rows}')
top_stock_list = list(top_stocks["Ticker"])
print(top_stock_list)


number of filtered tickers: 102
['CVNA', 'VKTX', 'CAVA', 'MSTR', 'NVDA', 'RKT', 'THC', 'VST', 'ALNY', 'COHR', 'SN', 'APP', 'COKE', 'NTRA', 'PLTR', 'IRM', 'HOOD', 'GDDY', 'VIRT', 'SPOT', 'NU', 'DKS', 'CRUS', 'UI', 'HWM', 'NRG', 'MMM', 'UHS', 'COIN', 'MHK', 'FICO', 'PPC', 'VNO', 'UWMC', 'WING', 'BURL', 'KKR', 'TPL', 'TRGP', 'WAL', 'JEF', 'MPWR', 'AXON', 'VNOM', 'TOL', 'GAP', 'NTAP', 'FRPT', 'EVR', 'JLL', 'UTHR', 'GE', 'DOCS', 'AVGO', 'RARE', 'HIW', 'DHI', 'GWRE', 'ANET', 'K', 'CFG', 'PGR', 'LAZ', 'GME', 'TSM', 'NEM', 'LLY', 'FYBR', 'CBRE', 'PHM', 'HRB', 'ALSN', 'PSTG', 'TPG', 'LPX', 'ISRG', 'BFAM', 'GRMN', 'EME', 'FIX', 'TRU', 'RCL', 'PSN', 'NVR', 'FITB', 'GM', 'HCA', 'RBLX', 'EFX', 'ALLY', 'SYF', 'VTR', 'CTAS', 'ATI', 'MSI', 'DVA', 'VRT', 'LII', 'TT', 'URI', 'META', 'MTZ']


In [ ]:
# @title Filtered Stocks with Increasing RS > 100
filtered_rank_stock = filter_increasing_relative_strength(rank_stock)
data_table.DataTable(filtered_rank_stock, include_index=False, num_rows_per_page=10)

,Ticker,Price,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
927,RKT,20.95,Financial Services,Mortgage Finance,160.81,132.12,122.77,111.69,99,96,92,81,6
816,COKE,1353.90,Consumer Defensive,Beverages—Non-Alcoholic,141.99,131.85,115.48,107.68,98,96,87,75,13
237,VIRT,30.74,Financial Services,Capital Markets,136.58,123.62,113.00,79.90,98,93,84,13,19
193,UI,206.38,Technology,Communication Equipment,134.03,123.66,96.19,68.59,97,93,53,3,24
501,MMM,130.91,Industrials,Conglomerates,131.77,128.58,109.16,79.05,97,95,79,11,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...
906,FE,43.47,Utilities,Utilities—Regulated Electric,100.81,94.02,91.85,82.45,61,44,41,18,393
343,DTE,124.71,Utilities,Utilities—Regulated Electric,100.49,95.42,92.56,84.54,60,48,43,23,398
579,SWK,100.96,Industrials,Tools & Accessories,100.34,92.68,86.65,85.23,60,39,26,25,401
641,WEC,92.93,Utilities,Utilities—Regulated Electric,100.29,86.82,86.30,76.43,60,23,25,8,404


### RS Rating and Ranking for Taiwan Stocks

In [13]:
from vistock import tw

source = "上市+上櫃" #@param ["上市", "上櫃", "上市+上櫃", "興櫃", "全部"]
period = "2y" # @param ["6mo","1y","ytd","2y"]

code_from_name = {
    '上市': 'TWSE',
    '上櫃': 'TPEX',
    '上市+上櫃': 'TWSE+TPEX',
    '興櫃': 'ESB',
    '全部': 'TWSE+TPEX+ESB'
}

tw_stocks, tw_industries = rank(code_from_name[source], period,
     tickers_getter=tw.get_tickers, ticker_ref='^TWII')
for df in (tw_stocks, tw_industries):
    display(data_table.DataTable(df, include_index=False, num_rows_per_page=10))

[************          25%                       ]  512 of 2076 completedERROR:yfinance:Failed to get ticker '00850.TW' reason: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)
[*********************100%***********************]  2076 of 2076 completed
ERROR:yfinance:
69 Failed downloads:
ERROR:yfinance:['4771.TW', '1563.TW', '8487.TW', '00941.TW', '6423.TW', '00943.TW', '6928.TW', '4949.TW', '6914.TW', '00946.TW', '6794.TW', '6771.TW', '6949.TW', '00944.TW', '00945B.TW', '00940.TW', '8162.TW', '00939.TW']: YFInvalidPeriodError("%ticker%: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']")
ERROR:yfinance:['02001L.TW', '020037.TW', '020038.TW', '020031.TW', '020029.TW', '020018.TW', '02001S.TW']: YFInvalidPeriodError("%ticker%: Period '2y' is invalid, must be one of ['1d', '5d']")
ERROR:yfinance:['6890.TW', '00949.TW', '00947.TW', '3150.TW', '6957.TW', '00951.TW']: YFInvalidPeriodError("%ticker%: Period

..................................................


***
Your "TWSE+TPEX_stocks_2y_20240827.csv" is in the "out" folder.
Your "TWSE+TPEX_industries_2y_20240827.csv" is in the "out" folder.
***



,Ticker,Name,Price,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
1135,6144,得利影,98.50,Communication Services,Entertainment,502.53,454.28,149.16,92.40,97,97,92,46,1
869,6640,均華,999.00,Technology,Semiconductors,424.46,416.15,237.35,236.22,97,97,96,96,2
1904,1799,易威,218.00,Healthcare,Drug Manufacturers—Specialty & Generic,419.97,325.99,258.08,202.10,97,97,96,95,3
1687,6442,光聖,448.50,Technology,Electronic Components,374.24,409.51,183.52,213.00,97,97,95,95,4
2046,8374,羅昇,138.00,Industrials,Industrial Distribution,334.70,334.99,123.40,82.34,97,97,87,14,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383,3701,大眾控,40.20,Technology,Computer Hardware,56.25,62.45,73.00,93.95,0,0,12,50,2071
898,6958,日盛台駿,31.15,Financial Services,Credit Services,56.11,74.24,74.05,86.88,0,11,14,28,2072
528,2740,天蔥,NaN,Consumer Cyclical,Restaurants,55.93,59.12,68.85,79.09,0,0,6,9,2073
1260,5227,立凱-KY,29.65,Industrials,Electrical Equipment & Parts,55.29,65.63,56.76,70.41,0,1,0,1,2074


,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
84,Industrial Distribution,Industrials,147.07,149.27,94.92,88.50,"8374,911608,3114,2373",99,98,72,29,1
56,Utilities—Renewable,Utilities,137.05,137.70,100.34,115.09,"6869,6873,6806,8087",98,97,81,96,2
40,Real Estate—Development,Real Estate,132.80,136.28,120.62,106.47,"2524,5508,3188,2537,1436,5455,2718,3489,6171,4...",97,96,97,84,3
49,Entertainment,Communication Services,132.77,126.12,92.15,93.89,"6144,4806,6596,8446,8450,8487,6184,6625,6464,6856",96,93,65,57,4
57,Recreational Vehicles,Consumer Cyclical,126.78,101.28,77.31,97.41,"8937,9110,8478",95,84,13,67,5
...,...,...,...,...,...,...,...,...,...,...,...,...
10,Credit Services,Financial Services,77.21,76.51,73.74,85.62,"9941A,6592B,6592A,6592,5871,9941,6958",4,3,6,14,91
24,Auto Manufacturers,Consumer Cyclical,74.33,76.68,79.62,93.23,"1599,2258,2206,2201,2227,2204",3,4,19,55,92
71,Pharmaceutical Retailers,Healthcare,73.54,71.55,70.03,85.84,"4173,4175,6469",2,0,3,15,93
86,Travel Services,Consumer Cyclical,73.02,79.73,94.11,84.43,"2745,2731,5706,2743,2719,6242,2734",1,12,68,9,94


In [14]:
# @title Top Percentile Taiwan Stocks
min_percentile = 95 # @param {"type":"slider","min":1,"max":99,"step":1}

top_stocks = tw_stocks[tw_stocks[ibd.TITLE_PERCENTILE] >= min_percentile]
num_rows, _ = top_stocks.shape
print(f'\nnumber of filtered tickers: {num_rows}')
top_stock_list = list(top_stocks["Ticker"])
top_stock_list = [tw.stock_name(ticker) for ticker in top_stock_list]
print(top_stock_list)


number of filtered tickers: 63
['得利影', '均華', '易威', '光聖', '羅昇', '福裕', '晶彩科', '錦明', '穎漢', '慧友', '均豪', '京城', '康全電訊', '皇昌', '昆盈', '擎亞', '福大', '欣巴巴', '志聖', '海悅', '新復興', '所羅門', '新門', '彬台', '高鋒', '太普高', '弘塑', '天方能源', '東捷', '弘憶股', '順藥', '天揚', '旺矽', '合騏', '翔耀', '福懋油', '天品', '鑫科', '精湛', '慶騰', '昇陽半導體', '系微', '永信建', '訊舟', '鏵友益', '花王', '雲豹能源', '力麗店', '鈊象', '世紀', '訊聯基因', '晟銘電', '印鉐', '泰金-KY', '鑫龍騰', '精材', '喬福', '聯鈞', '藝舍-KY', '德晉', '峰源-KY', '安國', '泓德能源-創']


In [15]:
# @title Filtered Taiwan Stocks with Increasing RS > 100

def convert_to_list(value):
    if isinstance(value, str):
        return value.split(',')
    return value

filtered_tw_stocks = filter_increasing_relative_strength(tw_stocks)

filtered_tw_industries = filter_increasing_relative_strength(tw_industries)
update_tickers_with_names(filtered_tw_industries, 'Tickers')
#filtered_tw_industries.loc[:, 'Tickers'] = filtered_tw_industries['Tickers'].apply(convert_to_list)

for df in (filtered_tw_stocks, filtered_tw_industries):
    display(data_table.DataTable(df, include_index=False, num_rows_per_page=10))

,Ticker,Name,Price,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
1135,6144,得利影,98.50,Communication Services,Entertainment,502.53,454.28,149.16,92.40,97,97,92,46,1
869,6640,均華,999.00,Technology,Semiconductors,424.46,416.15,237.35,236.22,97,97,96,96,2
1904,1799,易威,218.00,Healthcare,Drug Manufacturers—Specialty & Generic,419.97,325.99,258.08,202.10,97,97,96,95,3
1695,4513,福裕,78.90,Industrials,Specialty Industrial Machinery,299.55,178.23,152.50,96.35,97,95,93,55,6
330,3535,晶彩科,80.30,Technology,Computer Hardware,298.84,290.85,122.94,107.60,97,97,87,73,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1627,00692,富邦公司治理,42.30,NaN,NaN,102.37,101.51,100.76,98.29,73,73,72,58,506
1576,00922,國泰台灣領袖50,21.31,NaN,NaN,101.36,101.04,99.93,95.99,72,73,71,54,530
136,0055,元大MSCI金融,28.93,NaN,NaN,101.04,96.56,89.66,89.64,72,67,56,38,535
1642,006204,永豐臺灣加權,NaN,NaN,NaN,101.01,98.89,98.45,98.28,72,70,70,58,536


,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
54,Insurance—Life,Financial Services,102.33,93.72,86.23,84.65,"新光金,三商壽,富邦金,三商,新光金甲特,新光金乙特,國泰金,開發金,開發金乙特,國泰特,富...",79,62,47,10,19


### Remove files in GitHub Repository

In [16]:
# @title CSV Deleter
import re
import ipywidgets as widgets

# Example filenames
with requests_cache.disabled():
    all_filenames = github.list_filenames()

# Function to extract unique dates from filenames
def extract_dates(filenames):
    date_pattern = r'\d{8}'
    dates = set()
    for fn in filenames:
        match = re.search(date_pattern, fn)
        if match:
            dates.add(match.group(0))
    return sorted(dates, reverse=True)  # Sort dates from newest to oldest

# Function to remove a file (replace with your actual implementation)
def remove_file(filename):
    print(f"Removing file: {filename}")
    with requests_cache.disabled():
        github.remove_file(filename)
    all_filenames.remove(filename)

#-------------------------------------------------------------------------------

# Update file selector options based on selected date
def update_file_selector(change):
    def selector_width(filenames):
        max_filename_length = max(len(fn) for fn in filenames)
        return f'{max_filename_length * 10}px'  # 10px width per character

    selected_date = change['new']
    lst_fns = [fn for fn in all_filenames if selected_date in fn]
    file_selector_widget.options = lst_fns
    file_selector_widget.rows = len(file_selector_widget.options)
    file_selector_widget.layout=widgets.Layout(width=selector_width(lst_fns))

# Function to delete selected files
def delete_files(button):
    selected_files = file_selector_widget.value
    for file in selected_files:
        remove_file(file)
    update_widgets()    # Update widgets after deletion

# Update widgets to reflect current state
def update_widgets():
    # Refresh the date selector
    dates = extract_dates(all_filenames)
    selected_date = date_selector_widget.value
    if dates and selected_date not in dates:
        i = date_selector_widget.options.index(selected_date)
        if i > len(dates) - 1:
            selected_date = dates[-1]
        else:
            selected_date = dates[i]
    date_selector_widget.options = dates
    date_selector_widget.value = selected_date
    if not dates:
        file_selector_widget.options = []
        return
    update_file_selector({'new': date_selector_widget.value})

#-------------------------------------------------------------------------------

# Create a widget for selecting dates
def create_date_selector(dates):
    return widgets.Dropdown(
        options=dates,
        value = dates[0] if dates else None,
        description='Date:',
        disabled=False
    )

# Create a widget for selecting files
def create_file_selector(filenames):

    return widgets.SelectMultiple(
        options=[],
        value=[],
        description='Files',
        disabled=False,
    )

# Create widgets
dates = extract_dates(all_filenames)
date_selector_widget = create_date_selector(dates)
file_selector_widget = create_file_selector(all_filenames)
delete_button = widgets.Button(description="Delete Selected Files")
delete_button.on_click(delete_files)

# Initialize the file selector with the latest date
update_widgets()

# Set up the observer to update file selector when date is changed
date_selector_widget.observe(update_file_selector, names='value')

# Display widgets
display(date_selector_widget)
display(file_selector_widget)
display(delete_button)


Dropdown(description='Date:', options=('20240827', '20240826'), value='20240827')

SelectMultiple(description='Files', layout=Layout(width='360px'), options=('SOX_industries_2y_20240827.csv', '…

Button(description='Delete Selected Files', style=ButtonStyle())

Removing file: DJIA_industries_2y_20240826.csv
Removing file: DJIA_stocks_2y_20240826.csv
Removing file: NDX_industries_2y_20240826.csv
Removing file: NDX_stocks_2y_20240826.csv
Removing file: RUI_industries_2y_20240826.csv
Removing file: RUI_stocks_2y_20240826.csv
Removing file: SOX_industries_2y_20240826.csv
Removing file: SOX_stocks_2y_20240826.csv
Removing file: SPX+DJIA+NDX+RUI+SOX_industries_2y_20240826.csv
Removing file: SPX+DJIA+NDX+RUI+SOX_stocks_2y_20240826.csv
Removing file: SPX_industries_2y_20240826.csv
Removing file: SPX_stocks_2y_20240826.csv
Removing file: TPEX_industries_2y_20240826.csv
Removing file: TPEX_stocks_2y_20240826.csv
Removing file: TWSE+TPEX_industries_2y_20240826.csv
Removing file: TWSE+TPEX_stocks_2y_20240826.csv
Removing file: TWSE_industries_2y_20240826.csv
Removing file: TWSE_stocks_2y_20240826.csv
